In [1]:
# Import library:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

# Définition du chemin d'accès:
### Céline
%cd C:\Users\c.doussot\Desktop\Data Analyst\GitHub\ParisPyVelib_Datas
### Hermine
%cd C:\Users\h.berthon\Documents\GitHub\ParisPyVelib\ParisPyVelib_Datas
### Tarik
%cd C:\Users\Home\Documents\Git\ParisPyVelib\Data

C:\Users\c.doussot\Desktop\Data Analyst\GitHub\ParisPyVelib_Datas


- Une idée serait de faire la modélisation sur les comptages horaires moyen par jour et non par heure. Cela simplifiera le nombre de données et permettra d'intégrer facilement les données météos.